In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")


# Pytorch device specific configuration
# Pytorch Gpu Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [ ]:
# Generate data
data = np.random.randn(50000, 2) * 100
labels = np.sum(data, axis=1)
print(f"Data Shape: {data.shape}, Label Shape: {labels.shape}")

Data Shape: (50000, 2), Label Shape: (50000,)


In [ ]:
# Convert data into tensors
dataT = torch.tensor(data).float()
labelsT = torch.tensor(labels).long()

# train test splits
train_data, test_data_temp, train_labels, test_labels_temp = train_test_split(
    dataT, labelsT, train_size=0.8
)
test_data, dev_data, test_labels, dev_labels = train_test_split(
    test_data_temp, test_labels_temp, train_size=0.5
)

# Pytorch datasets
train_dataT = TensorDataset(train_data, train_labels)
dev_dataT = TensorDataset(dev_data, dev_labels)
test_dataT = TensorDataset(test_data, test_labels)

# Dataloader objects
batchSize = 32
train_loader = DataLoader(
    train_dataT,
    batch_size=batchSize,
    shuffle=True,
    drop_last=True,
    generator=torch.Generator(device),
)
dev_loader = DataLoader(
    dev_dataT,
    batch_size=dev_dataT.tensors[0].shape[0],
    generator=torch.Generator(device),
)
test_loader = DataLoader(
    test_dataT,
    batch_size=test_dataT.tensors[0].shape[0],
    generator=torch.Generator(device),
)

In [29]:
# Function to create the class
def createTheModel():
    class Sum2NumbersClass(nn.Module):
        def __init__(self):
            super().__init__()

            # Input
            self.input = nn.Linear(2, 128)
            # Hidden Layers
            self.hidden1 = nn.Linear(128, 256)
            self.hidden2 = nn.Linear(256, 256)
            self.hidden3 = nn.Linear(256, 64)
            # Output Layers
            self.output = nn.Linear(64, 1)

        def forward(self, x):
            # Input
            x = F.relu(self.input(x))

            # Hidden Layers
            x = F.relu(self.hidden1(x))
            x = F.relu(self.hidden2(x))
            x = F.relu(self.hidden3(x))

            # Output Layers
            return self.output(x)

    net = Sum2NumbersClass()

    # Loss Function
    lossFun = nn.MSELoss()

    # Optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

    return net, lossFun, optimizer

In [36]:
# Test the model
net, lossFun, optimizer = createTheModel()
tempData = torch.tensor(100 * np.random.randn(100, 2)).float()
print(net(tempData).shape)

torch.Size([100, 1])


In [37]:
# epochs
nEpochs = 30
def trainTheModel():
    # Model Instance
    net, lossFun, optimizer = createTheModel()

    # Initialize variables
    trainAcc = []
    devAcc = []
    losses = np.zeros(nEpochs)

    # The loop
    for epoch in range(nEpochs):
        # Model training mode
        net.train()

        # Batch loss and accuracy initialization
        batchLoss = []
        batchAcc = []

        # The batch loop
        for X, y in train_loader:

            # Forward pass
            yHat = net(X)
            loss = lossFun(yHat, y)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Batch Loss and accuracy
            batchLoss.append(loss.cpu().item())
            batchAcc.append((100 * torch.mean((yHat == y))).cpu())

        # Batch loop end

        # Train Acc
        trainAcc.append(torch.mean(batchAcc))
        losses[epoch] = torch.mean(batchLoss)

        # Dev accuracy
        net.eval()
        X, y = next(iter(dev_loader))
        with torch.no_grad():
            yHat = net(X)
        devAcc.append((100 * torch.mean((yHat == y))).cpu())

    # The Loop End

    return trainAcc, devAcc, losses